In [0]:
import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/Build-Week-AirBnB-Optimal-Price-2/Data-Science/JulieWang/data/cleaned_data.csv')

In [2]:
df.head()

,Unnamed: 0,accommodates,bathrooms,bedrooms,price,cleaning_fee,security_deposit,extra_people,guests_included,distance,size,host_is_superhost,neighbourhood_group_cleansed,latitude,longitude,property_type,room_type,bed_type,amenities,minimum_nights,instant_bookable,is_business_travel_ready,cancellation_policy,tv,wifi,kitchen,washer,dryer,heating,free parking,smoking allowed
0,0,3,1.0,1.0,60.0,30.0,200.0,28.0,1,1.624906,75.0,True,Mitte,52.534537,13.402557,Guesthouse,Entire home/apt,Real Bed,"{tv,""cable tv"",wifi,kitchen,gym,heating,""famil...",4,False,False,strict_14_with_grace_period,1,1,1,1,1,1,0,0
1,1,2,1.0,1.0,17.0,0.0,0.0,0.0,1,3.170627,25.0,False,Pankow,52.548513,13.404553,Apartment,Private room,Real Bed,"{wifi,kitchen,elevator,heating,washer,essentia...",2,False,False,flexible,0,1,1,1,1,1,0,0
2,2,4,1.0,1.0,90.0,50.0,200.0,20.0,2,1.872072,68.0,False,Pankow,52.534996,13.417579,Apartment,Entire home/apt,Real Bed,"{internet,wifi,kitchen,""buzzer/wireless interc...",62,True,False,strict_14_with_grace_period,0,1,1,1,1,1,0,0
3,3,2,1.0,1.0,26.0,30.0,250.0,18.0,1,4.456289,26.0,False,Tempelhof - Schöneberg,52.498855,13.349065,Apartment,Private room,Pull-out Sofa,"{internet,wifi,""pets allowed"",""pets live on th...",5,False,False,strict_14_with_grace_period,0,1,0,1,1,1,0,0
4,4,2,1.0,1.0,42.0,0.0,0.0,24.0,1,2.663910,20.0,True,Pankow,52.543157,13.415091,Apartment,Private room,Real Bed,"{wifi,heating,""family/kid friendly"",essentials...",2,False,False,moderate,0,1,0,0,1,1,0,0


In [17]:
df['bed_type'].value_counts()

Real Bed         21589
Pull-out Sofa      448
Futon              237
Couch               69
Airbed              23
Name: bed_type, dtype: int64

In [0]:
df['neighbourhood'] = df['neighbourhood_group_cleansed'].replace({'Friedrichshain-Kreuzberg':0, 'Mitte':1, 'Pankow':2, 'Neukölln':3,
                                                                  'Charlottenburg-Wilm.':4, 'Tempelhof - Schöneberg':5, 'Lichtenberg':6,
                                                                  'Treptow - Köpenick':7, 'Steglitz - Zehlendorf':8, 'Reinickendorf':9,
                                                                  'Marzahn - Hellersdorf':10, 'Spandau':11})

In [0]:
df['room_type_cleaned'] = df['room_type'].replace({'Entire home/apt':1, 'Private room':0, 'Shared room':2})
    
#     engineering cancelation policy
df['cancellation_policy_cleaned'] = df['cancellation_policy'].replace({'strict_14_with_grace_period':0, 'flexible':1,
                                                                      'moderate':2, 'super_strict_30':3, 
                                                                       'super_strict_60':4})

df['bed_type_cleaned'] = df['bed_type'].replace({'Real Bed':0, 'Pull-out Sofa':1, 'Futon':1, 'Couch':2, 'Airbed':2})

In [0]:
from collections import ChainMap

def clean_dataframe(df):
    
    df = df.copy()
    
#     keeping the columns we need
    columns = ['host_is_superhost', 'beds', 'price', 'security_deposit', 'cleaning_fee', 
    'property_type', 'minimum_nights', 'extra_people', 'latitude', 'longitude',
    'accommodates', 'bathrooms', 'bedrooms', 'cancellation_policy', 'room_type',
    'guests_included', 'amenities', 'description']

    df = df[columns]
    
#     stripping $ and ,
    df['cleaning_fee_cleaned'] = df['cleaning_fee'].replace({'[\$,]': ''}, regex=True).astype(float)
    df['security_deposit_cleaned'] = df['security_deposit'].replace('[\$,]', '', regex=True).astype(float)
    df['price_cleaned'] = df['price'].replace({'[\$,]': ''}, regex=True).astype(float)
    df['extra_people_cleaned'] = df['extra_people'].replace({'[\$,]': ''}, regex=True).astype(float)
    
#     filling the NaN's
    df['cleaning_fee_filled'] = df['cleaning_fee_cleaned'].fillna(0)
    df['security_deposit_filled'] = df['security_deposit_cleaned'].fillna(0)

    #   adding columns for amenities

    # new_cols = ['cable_tv', 'wifi', 'pet_allowed', 'washer_and_dryer', 'kitchen_available', 'smoking_allowed']
    # for col in new_cols:
    #   df[col] = False

    # for i in range(len(df)):
    #   if 'Cable TV' in df.iloc[i]['amenities']:
    #     df.at[i,'cable_tv'] = True
    #   if 'Wifi' in df.iloc[i]['amenities']:
    #     df.at[i,'wifi'] = True
    #   if 'Pets allowed' in df.iloc[i]['amenities']:
    #     df.at[i,'pet_allowed'] = True
    #   if 'Washer' and 'Dryer' in df.iloc[i]['amenities']:
    #     df.at[i,'washer_and_dryer'] = True
    #   if 'Kitchen' in df.iloc[i]['amenities']:
    #     df.at[i,'kitchen_available'] = True
    #   if 'Free parking on premises' in df.iloc[i]['amenities']:
    #     df.at[i,'free_parking_on_premises'] = True
    #   if 'Smoking allowed' in df.iloc[i]['amenities']:
    #     df.at[i,'smoking_allowed'] = True

# extracting size from description column
    # df['property_size'] = df['description'].str.extract('(\d{2,3}\s?[smSM])', expand=True)
    # df['property_size'] = df['property_size'].replace(regex=True,to_replace=r'\D',value=r'')



#     dropping NaN's
    df = df.dropna()
    
#     engineering new columns
    list = ['t']
    df['superhost'] = df['host_is_superhost'].isin(list)
    
#   engineering  property type
    guesthouse = ['Guest suite', 'Guesthouse']
    apartment = ['Apartment', 'Serviced apartment', 'Aparthotel']
    condo = ['Townhouse', 'Loft', 'Condominium']
    house = ['House', 'Bed and breakfast', 'Bungalow', 'Villa', 'Tiny house', 'Cabin', 'Cottage', 'Chalet']
    other = ['Hostel', 'Other', 'Boat', 'Houseboat', 'Boutique hotel', 'Camper/RV', 'Hotel', 
             'Resort', 'Train', 'Casa particular (Cuba)', 'In-law', 'Cave', 'Island']


    d = ChainMap(dict.fromkeys(guesthouse, 0), 
                 dict.fromkeys(apartment, 1), 
                 dict.fromkeys(condo, 2), 
                 dict.fromkeys(house, 3),
                 dict.fromkeys(other, 4))
    df['property_type_cleaned'] = df['property_type'].map(d.get).astype(int)
    
#     engineeering room type
    df['room_type_cleaned'] = df['room_type'].replace({'Entire home/apt':1, 'Private room':0, 'Shared room':2}).astype(int)
    
#     engineering cancelation policy
    df['cancellation_policy_cleaned'] = df['cancellation_policy'].replace({'strict_14_with_grace_period':0, 'flexible':1,
                                                                      'moderate':2, 'super_strict_30':3, 
                                                                       'super_strict_60':4}).astype(int)
    
#     turning columns into int
    df['beds'] = df['beds'].astype(int)
    df['bedrooms'] = df['bedrooms'].astype(int)
    
#     dropping outliers in columns
    df = df[(df['price_cleaned'] != 0.0) & (df['price_cleaned'] < 1000.0)]
    df = df[df['beds'] <= 12]
    df = df[df['bedrooms'] <= 8]
    df = df[df['bathrooms'] <= 5.0]

    # df['property_size'] = df['property_size'].astype(int)
    # df['free_parking_on_premises'] = df['free_parking_on_premises'].astype(bool)

#     dropping extra columns
    drop_columns = ['cleaning_fee_cleaned', 'security_deposit_cleaned', 'cleaning_fee', 'security_deposit', 
                    'price', 'extra_people', 'host_is_superhost', 'property_type', 'room_type', 
                    'cancellation_policy', 'amenities', 'description']
    
    df = df.drop(columns=drop_columns)

#   remove the NaN's
    # df = df.dropna(axis=0)
    
    
    return df

In [0]:
from collections import ChainMap
guesthouse = ['Guest suite', 'Guesthouse']
apartment = ['Apartment', 'Serviced apartment', 'Aparthotel']
condo = ['Townhouse', 'Loft', 'Condominium']
house = ['House', 'Bed and breakfast', 'Bungalow', 'Villa', 'Tiny house', 'Cabin', 'Cottage', 'Chalet']
other = ['Hostel', 'Other', 'Boat', 'Houseboat', 'Boutique hotel', 'Camper/RV', 'Hotel', 
             'Resort', 'Train', 'Casa particular (Cuba)', 'In-law', 'Cave', 'Island']


d = ChainMap(dict.fromkeys(guesthouse, 0), 
                 dict.fromkeys(apartment, 1), 
                 dict.fromkeys(condo, 2), 
                 dict.fromkeys(house, 3),
                 dict.fromkeys(other, 4))
df['property_type_cleaned'] = df['property_type'].map(d.get)

In [12]:
df['property_type_cleaned'].value_counts()

1.0    20273
2.0     1153
3.0      509
4.0      289
0.0      131
Name: property_type_cleaned, dtype: int64

In [19]:
df.shape

(22366, 36)

In [20]:
df.isnull().sum()

Unnamed: 0                       0
accommodates                     0
bathrooms                        0
bedrooms                         0
price                            0
cleaning_fee                     0
security_deposit                 0
extra_people                     0
guests_included                  0
distance                         0
size                             0
host_is_superhost                0
neighbourhood_group_cleansed     0
latitude                         0
longitude                        0
property_type                    0
room_type                        0
bed_type                         0
amenities                        0
minimum_nights                   0
instant_bookable                 0
is_business_travel_ready         0
cancellation_policy              0
tv                               0
wifi                             0
kitchen                          0
washer                           0
dryer                            0
heating             

In [21]:
df = df.drop(columns=['property_type', 'neighbourhood_group_cleansed', 'room_type', 'bed_type',
                      'Unnamed: 0', 'amenities', 'cancellation_policy'])
df.head()

,accommodates,bathrooms,bedrooms,price,cleaning_fee,security_deposit,extra_people,guests_included,distance,size,host_is_superhost,latitude,longitude,minimum_nights,instant_bookable,is_business_travel_ready,tv,wifi,kitchen,washer,dryer,heating,free parking,smoking allowed,property_type_cleaned,neighbourhood,room_type_cleaned,cancellation_policy_cleaned,bed_type_cleaned
0,3,1.0,1.0,60.0,30.0,200.0,28.0,1,1.624906,75.0,True,52.534537,13.402557,4,False,False,1,1,1,1,1,1,0,0,0.0,1,1,0,0
1,2,1.0,1.0,17.0,0.0,0.0,0.0,1,3.170627,25.0,False,52.548513,13.404553,2,False,False,0,1,1,1,1,1,0,0,1.0,2,0,1,0
2,4,1.0,1.0,90.0,50.0,200.0,20.0,2,1.872072,68.0,False,52.534996,13.417579,62,True,False,0,1,1,1,1,1,0,0,1.0,2,1,0,0
3,2,1.0,1.0,26.0,30.0,250.0,18.0,1,4.456289,26.0,False,52.498855,13.349065,5,False,False,0,1,0,1,1,1,0,0,1.0,5,0,0,1
4,2,1.0,1.0,42.0,0.0,0.0,24.0,1,2.663910,20.0,True,52.543157,13.415091,2,False,False,0,1,0,0,1,1,0,0,1.0,2,0,2,0


In [22]:
df.shape

(22366, 29)

In [23]:
df.dtypes

accommodates                     int64
bathrooms                      float64
bedrooms                       float64
price                          float64
cleaning_fee                   float64
security_deposit               float64
extra_people                   float64
guests_included                  int64
distance                       float64
size                           float64
host_is_superhost                 bool
latitude                       float64
longitude                      float64
minimum_nights                   int64
instant_bookable                  bool
is_business_travel_ready          bool
tv                               int64
wifi                             int64
kitchen                          int64
washer                           int64
dryer                            int64
heating                          int64
free parking                     int64
smoking allowed                  int64
property_type_cleaned          float64
neighbourhood            

In [0]:
df['tv'] = df['tv'].replace({1:True, 0:False})
df['wifi'] = df['wifi'].replace({1:True, 0:False})
df['kitchen'] = df['kitchen'].replace({1:True, 0:False})
df['washer'] = df['washer'].replace({1:True, 0:False})
df['dryer'] = df['dryer'].replace({1:True, 0:False})
df['heating'] = df['heating'].replace({1:True, 0:False})
df['free parking'] = df['free parking'].replace({1:True, 0:False})
df['smoking allowed'] = df['smoking allowed'].replace({1:True, 0:False})

In [33]:
df.dtypes

accommodates                  int64
bathrooms                   float64
bedrooms                      int64
price                       float64
cleaning_fee                float64
security_deposit            float64
extra_people                float64
guests_included               int64
distance                    float64
size                        float64
host_is_superhost              bool
latitude                    float64
longitude                   float64
minimum_nights                int64
instant_bookable               bool
is_business_travel_ready       bool
tv                             bool
wifi                           bool
kitchen                        bool
washer                         bool
dryer                          bool
heating                        bool
free_parking                   bool
smoking_allowed                bool
property_type                 int64
neighbourhood                 int64
room_type                     int64
cancellation_policy         

In [0]:
df = df.dropna()

In [0]:
df['property_type_cleaned'] = df['property_type_cleaned'].astype(int)
df['bedrooms'] = df['bedrooms'].astype(int)

In [32]:
df = df.rename(columns={'property_type_cleaned':'property_type', 'room_type_cleaned':'room_type',
                        'bed_type_cleaned':'bed_type', 'cancellation_policy_cleaned':'cancellation_policy',
                        'free parking':'free_parking', 'smoking allowed':'smoking_allowed'})
df.head()

,accommodates,bathrooms,bedrooms,price,cleaning_fee,security_deposit,extra_people,guests_included,distance,size,host_is_superhost,latitude,longitude,minimum_nights,instant_bookable,is_business_travel_ready,tv,wifi,kitchen,washer,dryer,heating,free_parking,smoking_allowed,property_type,neighbourhood,room_type,cancellation_policy,bed_type
0,3,1.0,1,60.0,30.0,200.0,28.0,1,1.624906,75.0,True,52.534537,13.402557,4,False,False,True,True,True,True,True,True,False,False,0,1,1,0,0
1,2,1.0,1,17.0,0.0,0.0,0.0,1,3.170627,25.0,False,52.548513,13.404553,2,False,False,False,True,True,True,True,True,False,False,1,2,0,1,0
2,4,1.0,1,90.0,50.0,200.0,20.0,2,1.872072,68.0,False,52.534996,13.417579,62,True,False,False,True,True,True,True,True,False,False,1,2,1,0,0
3,2,1.0,1,26.0,30.0,250.0,18.0,1,4.456289,26.0,False,52.498855,13.349065,5,False,False,False,True,False,True,True,True,False,False,1,5,0,0,1
4,2,1.0,1,42.0,0.0,0.0,24.0,1,2.663910,20.0,True,52.543157,13.415091,2,False,False,False,True,False,False,True,True,False,False,1,2,0,2,0


In [34]:
df.isnull().sum()

accommodates                0
bathrooms                   0
bedrooms                    0
price                       0
cleaning_fee                0
security_deposit            0
extra_people                0
guests_included             0
distance                    0
size                        0
host_is_superhost           0
latitude                    0
longitude                   0
minimum_nights              0
instant_bookable            0
is_business_travel_ready    0
tv                          0
wifi                        0
kitchen                     0
washer                      0
dryer                       0
heating                     0
free_parking                0
smoking_allowed             0
property_type               0
neighbourhood               0
room_type                   0
cancellation_policy         0
bed_type                    0
dtype: int64

In [0]:
# df.to_csv('Airbnb data cleaned.csv')

In [0]:
from google.colab import files

df.to_csv('Airbnb data cleaned.csv')
files.download('Airbnb data cleaned.csv')